# 05 - Transformation - Geothermal Probes
### Author: Daniel Herrera-Russert
#### February 17, 2025

In [5]:
import pandas as pd
import geopandas as gpd
from scipy.spatial import cKDTree
import numpy as np
import os

# Set the default renderer for JupyterLab
pio.renderers.default = 'iframe'

## 1. Considerations

This chapter will focus on preparing the data for the geothermal potential model. We will analyze the spatial distribution of boreholes, assess their density, and evaluate their impact on geothermal energy extraction. The goal is to incorporate spatial constraints that account for borehole interference and define areas with higher geothermal potential.

Understanding borehole distribution and its impact on geothermal potential is critical due to several factors:
- **Thermal Interference**: Boreholes placed too close together can deplete the surrounding ground heat, reducing overall efficiency (Wagner, R. (2019). *Erdwärmesonden: Auslegung von Kleinanlagen mit Berücksichtigung von Nachbarsonden*. Amt für Hochbauten, Stadt Zürich.
  
- **Regulatory Guidelines**: Swiss regulations set minimum spacing requirements for borehole heat exchangers (BHE) to prevent excessive ground cooling and ensure efficiency. The Swiss standard SN 546 384/6:2010 by the Swiss Society of Engineers and Architects (SIA) provides guidelines but does not specify fixed distances, instead recommending that spacing be based on thermal interactions, ground conductivity, and system load. Earlier guidelines, such as AWP Merkblatt T1 (1992), suggested a minimum of 5 meters between BHEs to reduce thermal interference. [Source: SANNER-GEO.DE](https://sanner-geo.de/media/82adf22b1ec3dfecffff8010fffffff1.pdf?utm_source=chatgpt.com)
  
- **Energy Yield Optimization**: Identifying underutilized areas with low borehole density can maximize energy extraction efficiency (Fraunhofer IEG. (2023). *Deep Geothermal Energy and Borehole Systems*.

To model geothermal potential effectively, we will explore several spatial metrics that are expected to have an impact on energy yield:

**1. Borehole Density in a Given Area**
- Compute the number of boreholes within a defined radius (e.g., 100m) using spatial joins.
- Justify the radius choice based on:
  - **Thermal influence range**: Existing research and regulatory guidelines.
  - **Empirical data**: Correlating energy yield with borehole proximity.
  - **Regulatory constraints**: Minimum spacing rules from Swiss authorities.
- Test multiple radii (50m, 100m, 150m) to determine the most effective threshold.

**2. Nearest Borehole Distance**
- Calculate the shortest distance between each borehole and its nearest neighbor.
- Analyze how this distance impacts energy yield.
- Implement **penalty functions** for close proximity to prevent over-extraction in dense areas.

**3. Depth Interaction**
- Investigate whether closely spaced boreholes share similar depths.
- Penalize clusters where boreholes extract heat from the same depth range.
- Identify optimal depth distributions for maximizing energy efficiency.

**4. Identifying Optimal Areas for New Probes**
- Map regions where borehole density is low.
- Highlight locations where spacing and depth diversity are favorable for new installations.
- Use inverse density mapping techniques to identify high-potential zones.

### What are the Next Steps?
- Implement spatial joins and distance calculations using **GeoPandas** and **SciPy’s cKDTree**.
- Visualize borehole density and penalty functions with **Matplotlib** and **Seaborn**.
- Define and test models incorporating these constraints.


---

## 2. Borehole Densities

In [39]:
# Step 1: Load the borehole dataset and save as GeoPandas
geojson_path = "data/transformed/zh_geothermal_probes.geojson"
zh_geothermal_probes_gdf = gpd.read_file(geojson_path)
zh_geothermal_probes_gdf.head()

,gml_id,x,y,Waermeentnahme,Waermeeintrag,Sondentiefe,Gesamtsondenzahl,GBS-Nummer,Bohrprofil,lon,lat,geometry
0,erdwaermesonden.1464734,2684789,1248345,20.4,0.0,250,3.0,b 00-10370,0,8.561441,47.380481,POINT (2684788.902 1248344.823)
1,erdwaermesonden.1464735,2684962,1248289,68.0,0.0,250,10.0,b 00-10371,1,8.563721,47.379955,POINT (2684961.902 1248288.82)
2,erdwaermesonden.1464736,2684633,1248535,12.5,6.0,250,2.0,b 00-10287,0,8.559411,47.382209,POINT (2684632.902 1248534.826)
3,erdwaermesonden.1464737,2684626,1248526,12.5,6.0,250,2.0,b 00-10288,0,8.559317,47.382129,POINT (2684625.901 1248525.826)
4,erdwaermesonden.1464738,2684623,1248519,12.5,6.0,250,2.0,b 00-10289,0,8.559276,47.382067,POINT (2684622.901 1248518.826)


In [40]:
zh_geothermal_probes_gdf = zh_geothermal_probes_gdf.drop(columns=["x", "y"], errors="ignore")

In [41]:
zh_geothermal_probes_gdf.head()

,gml_id,Waermeentnahme,Waermeeintrag,Sondentiefe,Gesamtsondenzahl,GBS-Nummer,Bohrprofil,lon,lat,geometry
0,erdwaermesonden.1464734,20.4,0.0,250,3.0,b 00-10370,0,8.561441,47.380481,POINT (2684788.902 1248344.823)
1,erdwaermesonden.1464735,68.0,0.0,250,10.0,b 00-10371,1,8.563721,47.379955,POINT (2684961.902 1248288.82)
2,erdwaermesonden.1464736,12.5,6.0,250,2.0,b 00-10287,0,8.559411,47.382209,POINT (2684632.902 1248534.826)
3,erdwaermesonden.1464737,12.5,6.0,250,2.0,b 00-10288,0,8.559317,47.382129,POINT (2684625.901 1248525.826)
4,erdwaermesonden.1464738,12.5,6.0,250,2.0,b 00-10289,0,8.559276,47.382067,POINT (2684622.901 1248518.826)


In [44]:
# Extract coordinate tuples directly from geometry
coords = np.array([(point.x, point.y) for point in zh_geothermal_probes_gdf.geometry])
# Create KDTree for efficient spatial queries
tree = cKDTree(coords)

In [48]:
# Define search radii (in meters)
radii = [50, 100, 150]
for r in radii:
    print(f"Computing radius {r}m ...")
    zh_geothermal_probes_gdf[f'count_{r}m'] = [len(tree.query_ball_point(coord, r)) - 1 for coord in coords]
    print(f"✔")

# Save the updated dataset
output_path = "data/transformed/zh_geothermal_probes_with_density.geojson"
zh_geothermal_probes_gdf.to_file(output_path, driver="GeoJSON")

# Display first few rows
zh_geothermal_probes_gdf.head()

Computing radius 50m ...
✔
Computing radius 100m ...
✔
Computing radius 150m ...
✔


,gml_id,Waermeentnahme,Waermeeintrag,Sondentiefe,Gesamtsondenzahl,GBS-Nummer,Bohrprofil,lon,lat,geometry,count_50m,count_100m,count_150m
0,erdwaermesonden.1464734,20.4,0.0,250,3.0,b 00-10370,0,8.561441,47.380481,POINT (2684788.902 1248344.823),3,6,9
1,erdwaermesonden.1464735,68.0,0.0,250,10.0,b 00-10371,1,8.563721,47.379955,POINT (2684961.902 1248288.82),0,1,4
2,erdwaermesonden.1464736,12.5,6.0,250,2.0,b 00-10287,0,8.559411,47.382209,POINT (2684632.902 1248534.826),5,7,10
3,erdwaermesonden.1464737,12.5,6.0,250,2.0,b 00-10288,0,8.559317,47.382129,POINT (2684625.901 1248525.826),5,7,9
4,erdwaermesonden.1464738,12.5,6.0,250,2.0,b 00-10289,0,8.559276,47.382067,POINT (2684622.901 1248518.826),5,7,9
